# テスト Mass-Damper-Spring

# Motion Flow構築

In [ ]:
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
flow.load_motion_profile()
step_ctrl_index = 3
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=step_ctrl_index)
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)

In [ ]:
pprint.pprint(type(flow.discrete_time))
pprint.pprint(type(flow.mprof))
pprint.pprint(type(flow.controller))
pprint.pprint(type(flow.plant))
pprint.pprint(type(flow.plant.physical_obj))

# MDSPhysicalObject 属性確認

In [ ]:
pprint.pprint(flow.plant.physical_obj.get_config())
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")

# MDSPhysicalObject 属性変更

In [ ]:
flow.plant.physical_obj.mass = 3.0
flow.plant.physical_obj.damper=1.5
flow.plant.physical_obj.spring=2.5
flow.plant.physical_obj.spring_balance_pos=0.1
flow.plant.physical_obj.static_friction_coeff=0.21
flow.plant.physical_obj.dynamic_friction_coeff=0.11
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")

# プラント微分方程式 解曲線の確認（初期値問題を解く）

In [ ]:
import pprint
import tkmotion as tkm
flow = tkm.MotionFlow()
flow.load_discrete_time()
# デフォルトのプロファイル(指令速度0, 指令位置0)をロード
flow.load_motion_profile(filepath="tkmotion/prof/default_motion_prof_config.json", prof_index=0)
# デフォルトのコントローラをロード
flow.load_controller(filepath="tkmotion/ctrl/default_controller_config.json", ctrl_index=0)
# MDSPhysicalObjectのプラントをロード
plant_index=1
phyobj_index=0
flow.load_plant(filepath="tkmotion/plant/default_plant_config.json", plant_index=plant_index, phyobj_index=phyobj_index)
# 各コンポーネントの型を確認
pprint.pprint(type(flow.discrete_time))
pprint.pprint(type(flow.mprof))
pprint.pprint(type(flow.controller))
pprint.pprint(type(flow.plant))
pprint.pprint(type(flow.plant.physical_obj))
# 物理オブジェクトのパラメータを確認
pprint.pprint(f"{flow.plant.physical_obj.mass=}")
pprint.pprint(f"{flow.plant.physical_obj.damper=}")
pprint.pprint(f"{flow.plant.physical_obj.spring=}")
pprint.pprint(f"{flow.plant.physical_obj.spring_balance_pos=}")
pprint.pprint(f"{flow.plant.physical_obj.static_friction_coeff=}")
pprint.pprint(f"{flow.plant.physical_obj.dynamic_friction_coeff=}")
# 物理オブジェクトの状態を確認
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")
# 物理オブジェクトの初期位置を変える
flow.plant.physical_obj.set_state(acc=0, vel=0, pos=0.05)
pprint.pprint(f"{flow.plant.physical_obj.acc=}")
pprint.pprint(f"{flow.plant.physical_obj.vel=}")
pprint.pprint(f"{flow.plant.physical_obj.pos=}")


In [ ]:
# モーションフロー実行
df = flow.execute()
df

In [ ]:
# プラントと物理オブジェクトの確認
import plotly.express as px
fig = px.line(
    df,
    x="time_s",
    y=["obj_velocity_m_s", "obj_position_m"],
    title="Motion Profile Execution Result")
fig.update_layout(
    yaxis2=dict(
        title="Position (m)",  # 第2Y軸のタイトル
        overlaying="y",        # 第1Y軸の上に重ねる
        side="right",          # 右側に表示
        showgrid=False         # オプション: グリッド線を非表示
    ),
)
for trace in fig.data:
    # トレース名（凡例名）が 'obj_position_m' であるかを確認
    if trace.name == "obj_position_m":
        trace.update(yaxis="y2")
        break
fig.update_yaxes(title_text="Velocity (m/s)", row=1, col=1)
fig.update_xaxes(title_text="Time (s)", row=1, col=1)
fig.show()

# サンプル：DataFrameに辞書データを追加

In [ ]:
sample_dict = {
    "column_a": [0, 1, 2], 
    "column_b": [3, 4, 5],
    }
sample_dict

In [ ]:
import pandas as pd
df = pd.DataFrame(sample_dict)
df

In [ ]:
sample_dict_2 = {
    "column_c": [6, 7, 8], 
    "column_d": [9, 10, 11],
    }
sample_dict_2

In [ ]:
for key, value in sample_dict_2.items():
    df[key] = value
df